In [15]:
from datetime import datetime
import json
import urllib
import urllib2

from bs4 import BeautifulSoup

In [16]:
movie_id = 'tt1596363'

url = 'http://www.imdb.com/title/' + movie_id + '/reviews?ref_=tt_ov_rt'

In [20]:
# def getUrlSoup(url):
#     # Get the page response.
#     response = urllib.urlopen(url)
#     # Read the page response into raw content.
#     content = response.read()
#     # Soup it.
#     soup = BeautifulSoup(content)
#     return soup


# soup = getUrlSoup(url)

In [21]:
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

def getUrlSoup(url,hdr):
    req1 = urllib2.Request(url, headers=hdr)
    page = urllib2.urlopen(req1)
    content = page.read()
    soup = BeautifulSoup(content)
    return soup
    
soup = extractHtmlAndSoup(url,hdr)

In [51]:
main_block = (soup.body.find('div', attrs={'class':'reviews'})
                       .find('div', attrs={'id':'tn15main'})
                       .find('div', attrs={'id':'tn15content'}))

In [52]:
num_pages = int(main_block.findAll('table')[1]
                          .find('td',attrs={'nowrap':'1'}).get_text().split(' of ')[1].rstrip(':'))

print num_pages

20


In [82]:
import re

reviews_block = re.split(r"<hr[^>]*>", str(main_block))

reviews = []

for r in reviews_block[1:-3]:
    
    soup_review = BeautifulSoup(r)
    soup_review_header = soup_review.find('div')
    review = dict()
    
    review['user_name'] = soup_review_header.findAll('a')[1].get_text()
    review['user_href'] = soup_review_header.findAll('a')[1]['href']
    
    # Try to get the rating.
    soup_imgs = soup_review_header.findAll('img')
    review['review_rating'] = None
    if len(soup_imgs) > 1:
        review['review_rating'] = soup_imgs[1]['alt'].split('/')[0]
    
    review['review_useful'] = soup_review_header.find('small').get_text().split('people')[0].rstrip().lstrip()
    review['review_content'] = soup_review.find('p').get_text()
    
    reviews.append(review)
    

print json.dumps(reviews[:2],indent=4)

[
    {
        "review_content": "\nComedy is tragedy + time, right? We're now over 7 years out from the\napex of the American financial crisis, which spiraled outward across\nthe world, and yet what has really changed? People are still making\nmillions/billions off the suffering of others, corporate control reigns\nsupreme, fraud is common and remains largely unknown, wealth continues\nto be ever more concentrated in the grasp of a few, and the remainder\nof the populace are treated as proverbial rats and made to feel uncouth\nshould they question the system and question not wanting to live their\nlives playing this sadistic game. Taking 2 pennies and selling them to\nsomeone for a hundred dollars remains a legal activity, just call those\npennies by a different name and suddenly it's okay to pass them off as\nfair market.It doesn't sound funny at all, but The Big Short succeeds in turning\nthis demented and corrupt circus into something improbably hilarious\nand probing. The power o

In [ ]:
reviews_block = (soup.body.find('div', attrs={'class': 'widget article reviews'})
                          .find('div', attrs={'class':'reviews'}))